In [1]:
import panel as pn
import hvplot.pandas
import hvplot.xarray
import matplotlib.pyplot as plt
import seaborn as sns
import holoviews as hv
from holoviews import opts

import xarray as xr
import pandas as pd
import numpy as np
from scipy.stats import linregress
from itertools import product

from bokeh.models.formatters import BasicTickFormatter, TickFormatter

In [2]:
tf = BasicTickFormatter(use_scientific=True, power_limit_high=0, precision=0)

In [3]:
import os
port_tunnel = int(os.environ['PORT2'])
print('++ INFO: Second Port available: %d' % port_tunnel)

++ INFO: Second Port available: 46137


***

In [41]:
a='16.3,32.2,48.1'
b = np.array([float(i) for i in a.split(',')])
c = roi_ts.TE.values

In [45]:
np.all(b == c)

True

ERROR:tornado.application:Uncaught exception GET /ws (::1)
HTTPServerRequest(protocol='http', host='localhost:46137', method='GET', uri='/ws', version='HTTP/1.1', remote_ip='::1')
Traceback (most recent call last):
  File "/data/SFIMJGC_HCP7T/Apps/envs/me_ets_2024a/lib/python3.9/site-packages/tornado/websocket.py", line 937, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
  File "/data/SFIMJGC_HCP7T/Apps/envs/me_ets_2024a/lib/python3.9/site-packages/tornado/web.py", line 3290, in wrapper
    return method(self, *args, **kwargs)
  File "/data/SFIMJGC_HCP7T/Apps/envs/me_ets_2024a/lib/python3.9/site-packages/bokeh/server/views/ws.py", line 149, in open
    raise ProtocolError("Token is expired.")
bokeh.protocol.exceptions.ProtocolError: Token is expired.


In [30]:
roi_ts   = xr.open_dataarray('../temp/roi_ts.nc')
ets_ts   = xr.open_dataarray('../temp/edge_ts.nc')
ets_fits = xr.open_dataarray('../temp/ets_stats.nc')

***

In [5]:
SBJ_LIST=['P3SBJ06']
RUN_LIST=['Events01','Events02']
ECHOES={'E01':16.3,'E02':32.2,'E03':48.1}
Nrois = 400
Nacqs = 215
ROI_LIST = ['R'+str(r).zfill(3) for r in range(Nrois)]

In [6]:
echo_label_combs = [i for i in product(ECHOES.keys(),repeat=2)]
echo_times_combs = [i for i in product(ECHOES.values(),repeat=2)]
echo_times_mults = np.array([i*j for i,j in echo_times_combs]).round(2)
print('++INFO: echo_label_combs =' , echo_label_combs)
print('++INFO: echo_times_comns =' , echo_times_combs)
print('++INFO: echo_times_mult  =' , echo_times_mults)

++INFO: echo_label_combs = [('E01', 'E01'), ('E01', 'E02'), ('E01', 'E03'), ('E02', 'E01'), ('E02', 'E02'), ('E02', 'E03'), ('E03', 'E01'), ('E03', 'E02'), ('E03', 'E03')]
++INFO: echo_times_comns = [(16.3, 16.3), (16.3, 32.2), (16.3, 48.1), (32.2, 16.3), (32.2, 32.2), (32.2, 48.1), (48.1, 16.3), (48.1, 32.2), (48.1, 48.1)]
++INFO: echo_times_mult  = [ 265.69  524.86  784.03  524.86 1036.84 1548.82  784.03 1548.82 2313.61]


***

In [7]:
sbj_sel   = pn.widgets.Select(name='Subject', options=SBJ_LIST, value=SBJ_LIST[0], width=150)
run_sel   = pn.widgets.Select(name='Run', options=RUN_LIST, value=RUN_LIST[0],width=150)
x_te_sel  = pn.widgets.Select(name='Echo ROI x [ms]', options=list(ECHOES.values()), value=list(ECHOES.values())[0], width=150)
y_te_sel  = pn.widgets.Select(name='Echo ROI x [ms]', options=list(ECHOES.values()), value=list(ECHOES.values())[0], width=150)
x_roi_sel = pn.widgets.Select(name='ROI x', options=ROI_LIST, value='R209', width=150)
y_roi_sel = pn.widgets.Select(name='ROI y', options=ROI_LIST, value='R008', width=150)
tr_slider = pn.widgets.IntSlider(name='TR',start=0, end=Nacqs-1, step=1, value=100, width=495)

In [8]:
scan_sel = pn.Row(pn.WidgetBox(sbj_sel,run_sel), pn.WidgetBox(x_roi_sel,x_te_sel), pn.WidgetBox(y_roi_sel, y_te_sel))
controls = pn.Column(scan_sel,tr_slider)

#### Functions to plot individual ROI Time series

In [9]:
def plot_roi_ts(roi,te,color):
    aux_ts = roi_ts.sel(ROI=roi, TE=te)
    return aux_ts.hvplot(c=color,width=1000, height=300, yformatter=tf).opts(toolbar='above')
@pn.depends(x_roi_sel,x_te_sel,tr_slider)
def plot_roi_x_ts(roi,te,tr,color='b'):
    return plot_roi_ts(roi,te,tr,color=color)
@pn.depends(y_roi_sel,y_te_sel,tr_slider)
def plot_roi_y_ts(roi,te,tr,color='g'):
    return plot_roi_ts(roi,te,tr,color=color)
@pn.depends(x_roi_sel,x_te_sel,y_roi_sel,y_te_sel,tr_slider)
def plot_roi_xy_ts(roi_x,te_x,roi_y,te_y,tr,):
    plot_title = '%s [%s ms] & %s [%s ms]' % (roi_x,te_x,roi_y,te_y)
    plot = plot_roi_ts(roi_x,te_x,'b')*plot_roi_ts(roi_y,te_y,'g')
    plot = plot* hv.VLine(tr).opts(line_width=0.5,line_color='k',line_dash='dashed')
    plot.opts(opts.Overlay(title=plot_title))
    return plot

In [10]:
def plot_roi_ts_me(roi,tr):
    all_tes = list(ECHOES.values())
    ts_e01 = roi_ts.sel(ROI=roi,TE=all_tes[0])
    ts_e02 = roi_ts.sel(ROI=roi,TE=all_tes[1])
    ts_e03 = roi_ts.sel(ROI=roi,TE=all_tes[2])
    plot = ts_e01.hvplot(width=500, height=200, label='E01', yformatter=tf) * ts_e02.hvplot(width=500, height=200, label='E02', yformatter=tf) * ts_e03.hvplot(width=500, height=200, label='E03')
    plot = plot * hv.VLine(tr).opts(line_width=0.5,line_color='k',line_dash='dashed')
    plot.opts(legend_cols=3, legend_position='top')
    plot.opts(opts.Overlay(title=''))
    return plot
@pn.depends(x_roi_sel,tr_slider)
def plot_roi_x_ts_me(roi_x,tr):
    return plot_roi_ts_me(roi_x,tr)
@pn.depends(y_roi_sel,tr_slider)
def plot_roi_y_ts_me(roi_y,tr):
    return plot_roi_ts_me(roi_y,tr)

#### Functions to plot individual ROI ME-Scatter Plot

In [11]:
def plot_roi_tr_scat(roi,tr):
    data = roi_ts.sel(ROI=roi,TR=tr)
    slope, intercept, r_value, p_value, std_err = linregress(data.TE.values,data.values)
    title = 'R2=%.2f' % (r_value*r_value)
    scat = data.hvplot.scatter(aspect='square', frame_width=200,yformatter=tf, xformatter=tf)
    plot = scat * hv.Slope.from_scatter(scat).opts(line_color='k',line_width=0.5, line_dash='dashed')
    plot = plot * hv.Text(data.TE.values[1],0.01,title)
    plot.opts(opts.Overlay(title=''))
    return plot

@pn.depends(x_roi_sel,tr_slider)
def plot_roi_x_tr_scat(roi,tr):
    return plot_roi_tr_scat(roi,tr)
@pn.depends(y_roi_sel,tr_slider)
def plot_roi_y_tr_scat(roi,tr):
    return plot_roi_tr_scat(roi,tr)

#### Functions to plot Edge Timeseries

In [12]:
@pn.depends(x_roi_sel,y_roi_sel,tr_slider)
def plot_ets(roi_x,roi_y,tr):
    edge_name = '-'.join([roi_x,roi_y])
    if edge_name not in ets_ts.Edge:
        edge_name = '-'.join([roi_y,roi_x])
    aux_ets   = ets_ts.sel(Edge=edge_name)
    plot = aux_ets.hvplot(x='TR',y='ETS',by='TEs',legend='top', width=1000, title='', height=300, yformatter=tf)
    plot = plot * hv.VLine(tr).opts(line_width=0.5,line_color='k',line_dash='dashed')
    return plot
@pn.depends(x_roi_sel,y_roi_sel,tr_slider)
def plot_ets_r2(roi_x,roi_y,tr):
    edge_name = '-'.join([roi_x,roi_y])
    if edge_name not in ets_ts.Edge:
        edge_name = '-'.join([roi_y,roi_x])
    aux = ets_fits.sel(Edge=edge_name,Metric='R2')
    aux.name = 'R2'
    plot = aux.hvplot(width=1000,height=300,c='k', yformatter=tf)
    plot = plot * hv.VLine(tr).opts(line_width=0.5,line_color='k',line_dash='dashed')
    return plot

In [13]:
@pn.depends(x_roi_sel,y_roi_sel,tr_slider)
def plot_ets_tr_scatter(roi_x,roi_y,tr):
    edge_name = '-'.join([roi_x,roi_y])
    if edge_name not in ets_ts.Edge:
        edge_name = '-'.join([roi_y,roi_x])
    aux = pd.DataFrame(echo_times_mults,columns=['TE*TE'])
    aux['ETS'] = ets_ts.sel(Edge=edge_name,TR=tr).values
    slope, intercept, r_value, p_value, std_err = linregress(aux['TE*TE'].values,aux['ETS'].values)
    scat = aux.hvplot.scatter(x='TE*TE',y='ETS', aspect='square', frame_width=250, yformatter=tf, xformatter=tf)
    plot = scat * hv.Slope.from_scatter(scat).opts(line_color='k',line_width=.5,line_dash='dashed')
    plot = plot * hv.Text(500, 1.5e-4,'R^2=%.2f' % (r_value*r_value))
    return plot

In [14]:
@pn.depends(x_roi_sel,y_roi_sel,x_te_sel,y_te_sel,tr_slider)
def plot_ets_2_echo(roi_x,roi_y,te_x,te_y,tr):
    edge_name = '-'.join([roi_x,roi_y])
    if edge_name not in ets_ts.Edge:
        edge_name = '-'.join([roi_y,roi_x])
    aux_ets   = ets_ts.sel(Edge=edge_name,TEs=str(te_x)+'x'+str(te_y))
    plot = aux_ets.hvplot(x='TR',y='ETS',by='TEs',legend='top', width=1000, height=300,title='', yformatter=tf)
    plot = plot * hv.VLine(tr).opts(line_width=0.5,line_color='r',line_dash='dashed')
    return plot

In [15]:
@pn.depends(x_roi_sel,x_te_sel,y_roi_sel,y_te_sel)
def plot_ets2r_scat(roi_x,te_x,roi_y,te_y):
    edge_name = '-'.join([roi_x,roi_y])
    if edge_name not in ets_ts.Edge:
        edge_name = '-'.join([roi_y,roi_x])
    aux_ets_ts   = ets_ts.sel(Edge=edge_name,TEs=str(te_x)+'x'+str(te_y)).values
    aux_ets_r2   = ets_fits.sel(Edge=edge_name,Metric='R2').values
    aux = pd.DataFrame([aux_ets_ts,aux_ets_r2]).T
    aux.columns=['SPC^2','Slope']
    aux.index.name='TR'
    slope, intercept, r_value, p_value, std_err = linregress(aux['SPC^2'].values,aux['Slope'].values)
    scat = aux.hvplot.scatter(x='SPC^2',y='Slope', aspect='square', s=2, yformatter=tf, xformatter=tf, frame_height=250, hover_cols='TR')
    plot = scat * hv.Slope.from_scatter(scat).opts(line_dash='dashed',line_width=0.5,line_color='k')
    plot = plot * hv.Text(aux['SPC^2'].quantile(.95),aux['Slope'].mean(),'R = %.2f' % (r_value*r_value))
    return plot

***
# Create Dashboard

In [16]:
dashbord = pn.Row(pn.Column(controls, 
                            pn.WidgetBox(plot_roi_x_ts_me,plot_roi_x_tr_scat),
                            pn.WidgetBox(plot_roi_y_ts_me,plot_roi_y_tr_scat)),
                  pn.Column(pn.WidgetBox('## ME-ets',pn.Row(plot_ets,plot_ets_tr_scatter)),
                            pn.WidgetBox('## ETS vs. R^2',pn.Column(pn.Row(plot_ets_2_echo,plot_ets2r_scat),plot_ets_r2))))
                  

In [17]:
dashbord_server = dashbord.show(port=port_tunnel, open=False)

Launching server at http://localhost:46137


In [21]:
dashbord_server.stop()

In [30]:
data.TE.values

array([16.3, 32.2, 48.1])